# Real-world network

In [1]:
import networkx as nx
from networkx.algorithms.centrality import *
import numpy as np
import operator
from analysis import *

In [2]:
g = nx.read_gml("./RealNetworks/london_transport.gml")

In [3]:
print(nx.info(g))

Graph with 369 nodes and 430 edges


<ipython-input-3-99dac2172e06>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))


# Check connectedness

In [4]:
nx.is_connected(g)

True

In [5]:
nx.number_connected_components(g)

1

# Get LCC if graph is disconnected

# Relabel it all if nodes are strings
- To be able to run all types of correlation together
- But put the graph relabled in `g_relabled`

In [6]:
from myTruss import mappingAndRelabeling

In [7]:
g_relabled = mappingAndRelabeling(g)

In [8]:
print(nx.info(g))
print(nx.info(g_relabled))

Graph with 369 nodes and 430 edges
Graph with 369 nodes and 430 edges


<ipython-input-8-632c4a2d19f6>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))
<ipython-input-8-632c4a2d19f6>:2: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g_relabled))


In [9]:
g = g_relabled

# Infer community structure with Louvain

In [10]:
nx.__version__

'2.8.4'

In [11]:
import pandas as pd
from os.path  import exists
import infomap as im

In [12]:
im.__version__

'2.3.0'

In [13]:
G = g

In [14]:
partition   = nx.algorithms.community.louvain_communities(G, seed = 0)
partition_Q = nx.algorithms.community.modularity(G, partition)

for seed in Bar("Progress", check_tty = False).iter(range(1,5)):
    partition_   = nx.algorithms.community.louvain_communities(G, seed = seed)
    partition_Q_ = nx.algorithms.community.modularity(G, partition_)
    
    if partition_Q_ > partition_Q:
        partition   = partition_
        partition_Q = partition_Q_

Progress |################################| 4/4


In [15]:
communities = { community_ID : set(community) for (community_ID, community) in enumerate(partition, start = 1)}

In [16]:
community_assignments = dict()
for (community_ID, community) in communities.items():
    for node in community:
        community_assignments[node] = community_ID

In [17]:
results_file = "./results/londontransport-modularity.csv"

if not exists(results_file):
    # Map Equation Centrality
    tree_file = "./results/londontransport-temp.tree"

    # use Infomap to write a tree file that we change to reflect the Louvain partition
    infomap = im.Infomap(silent = True, num_trials = 1, directed = False, two_level = True, seed = 42)
    infomap.add_networkx_graph(G)
    infomap.run()
    infomap.write_tree(tree_file)
    
    infomap_tree = dict()

    # read the tree file
    with open(tree_file, "r") as fh:
        for line in fh:
            if not line.startswith("#"):
                _, flow, name, node_id = line.strip().split(" ")
                name = name[1:-1]
                infomap_tree[int(name)] = (flow, node_id)
    
    # write the Louvain partition to an Infomap tree file
    with open("./results/londontransport-modularity.tree", "w") as fh:
        for community_ID, community in enumerate(partition, start = 1):
            for node_pos, node in enumerate(community, start = 1):
                path = f"{community_ID}:{node_pos}"
                flow, node_id = infomap_tree[node]
                fh.write(f"{path} {flow} \"{node}\" {node_id}\n")
    
    # Load the Louvain partition with Infomap and use unrecorded link teleportation
    infomap2 = im.Infomap("--silent --no-infomap --cluster-data ./results/londontransport-modularity.tree")
    node_mapping = infomap2.add_networkx_graph(G)
    infomap2.run()
    
    mec_ranks = [node for node, _score in sorted( [ (node_mapping[node.node_id], node.modular_centrality) for node in infomap2.iterLeafNodes() ]
                                                 , key     = second
                                                 , reverse = True
                                                 )]

    
    index = []
    for node in G.nodes:
        index.append(node)

    results = pd.DataFrame(index=index)

    results["map equation centrality rank"]       = toRanking(mec_ranks,        results.index)
    results.to_csv(results_file)

else:
    results = pd.read_csv(results_file, index_col = 0)

    mec_ranks         = mkRanks(results["map equation centrality rank"])

In [18]:
len(mec_ranks)

369

In [19]:
# node 107 has rank 0 (i.e. 1)
# node 1684 has rank 1 (i.e. 2)
# ...
# node 0 has rank 4 (i.e. 5)
mec_ranks

[54,
 69,
 5,
 27,
 89,
 80,
 1,
 4,
 77,
 78,
 108,
 24,
 32,
 198,
 17,
 9,
 150,
 87,
 224,
 30,
 64,
 28,
 66,
 142,
 75,
 290,
 178,
 136,
 35,
 149,
 152,
 154,
 70,
 222,
 225,
 14,
 112,
 115,
 131,
 132,
 134,
 15,
 133,
 169,
 25,
 26,
 155,
 156,
 292,
 71,
 121,
 38,
 39,
 267,
 153,
 223,
 270,
 18,
 272,
 107,
 110,
 111,
 113,
 278,
 294,
 352,
 184,
 326,
 329,
 21,
 20,
 340,
 98,
 322,
 229,
 45,
 196,
 138,
 139,
 140,
 141,
 31,
 210,
 23,
 92,
 88,
 90,
 91,
 157,
 221,
 171,
 94,
 194,
 195,
 68,
 197,
 193,
 67,
 72,
 144,
 146,
 82,
 96,
 161,
 97,
 293,
 243,
 244,
 245,
 52,
 53,
 65,
 59,
 60,
 61,
 62,
 63,
 262,
 263,
 73,
 74,
 76,
 264,
 81,
 215,
 216,
 218,
 348,
 349,
 350,
 289,
 291,
 302,
 304,
 305,
 180,
 181,
 118,
 0,
 2,
 3,
 7,
 137,
 207,
 209,
 148,
 345,
 347,
 29,
 162,
 34,
 37,
 164,
 163,
 36,
 151,
 286,
 300,
 301,
 310,
 79,
 351,
 363,
 238,
 239,
 265,
 266,
 268,
 269,
 335,
 336,
 275,
 276,
 273,
 274,
 271,
 41,
 43,
 40,
 361,

In [20]:
counter = len(mec_ranks)
dict_map_eq_centrality = {}

In [21]:
for node in mec_ranks:
    dict_map_eq_centrality[int(node)] = counter
    counter = counter - 1

In [22]:
dict_map_eq_centrality

{54: 369,
 69: 368,
 5: 367,
 27: 366,
 89: 365,
 80: 364,
 1: 363,
 4: 362,
 77: 361,
 78: 360,
 108: 359,
 24: 358,
 32: 357,
 198: 356,
 17: 355,
 9: 354,
 150: 353,
 87: 352,
 224: 351,
 30: 350,
 64: 349,
 28: 348,
 66: 347,
 142: 346,
 75: 345,
 290: 344,
 178: 343,
 136: 342,
 35: 341,
 149: 340,
 152: 339,
 154: 338,
 70: 337,
 222: 336,
 225: 335,
 14: 334,
 112: 333,
 115: 332,
 131: 331,
 132: 330,
 134: 329,
 15: 328,
 133: 327,
 169: 326,
 25: 325,
 26: 324,
 155: 323,
 156: 322,
 292: 321,
 71: 320,
 121: 319,
 38: 318,
 39: 317,
 267: 316,
 153: 315,
 223: 314,
 270: 313,
 18: 312,
 272: 311,
 107: 310,
 110: 309,
 111: 308,
 113: 307,
 278: 306,
 294: 305,
 352: 304,
 184: 303,
 326: 302,
 329: 301,
 21: 300,
 20: 299,
 340: 298,
 98: 297,
 322: 296,
 229: 295,
 45: 294,
 196: 293,
 138: 292,
 139: 291,
 140: 290,
 141: 289,
 31: 288,
 210: 287,
 23: 286,
 92: 285,
 88: 284,
 90: 283,
 91: 282,
 157: 281,
 221: 280,
 171: 279,
 94: 278,
 194: 277,
 195: 276,
 68: 275,
 

In [23]:
# Write Centrality
with open('./MapEquationCentralityResults/Infra-LondonTransport/dict_map_eq_centrality.txt', 'w') as f:
    for key, value in dict_map_eq_centrality.items():
        f.write('%s:%s\n' % (key, value))

In [24]:
# Read Centrality
dict_map_eq_centrality_read = dict()
with open('./MapEquationCentralityResults/Infra-LondonTransport/dict_map_eq_centrality.txt') as raw_data:
    for item in raw_data:
        if ':' in item:
            key,value = item.split(':', 1)
            value = value.replace('\n', '')
            dict_map_eq_centrality_read[int(key)]=float(value)
        else:
            pass # deal with bad lines of text here

In [25]:
dict_map_eq_centrality_read == dict_map_eq_centrality

True